In [4]:
import tensorflow as tf


a = tf.constant([1,2,3,4], name='a')
b = tf.constant([1,2,3,4], name='b')
c = a*b

sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run([a,b,c]))


[array([1, 2, 3, 4], dtype=int32), array([1, 2, 3, 4], dtype=int32), array([ 1,  4,  9, 16], dtype=int32)]


In [15]:
import numpy as np
import tensorflow as tf


x = tf.placeholder(tf.float32, name='x')
c = tf.constant(2.0, name='c')
cx_squared = c*x*x

sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

#print(sess.run(cx_squared, feed_dict={x:np.array([1,2,3])}))

print(sess.run(cx_squared, feed_dict={x:np.array([[1,2],[6,7]])}))


[[ 2.  8.]
 [72. 98.]]


In [2]:
import numpy as np
import tensorflow as tf



def readMat(path):
    return np.load(path)

def writeMat(path, m):
    np.save(path, m)


def loadData(dataPath, targetPath):
    # Loading my data
    inputData = readMat(dataPath)
    target = readMat(targetPath)

    trainData = inputData[:,0:70].T
    testData = inputData[:,70:].T

    trainTarget = np.expand_dims(target[0:70], 1)
    testTarget = np.expand_dims(target[70:],1)
    return trainData, trainTarget, testData, testTarget


def buildGraph():
    # Variable creation
    W = tf.Variable(tf.truncated_normal(shape=[5,1], stddev=0.5), name='weights')
    b = tf.Variable(0.0, name='biases')
    X = tf.placeholder(tf.float32, [None, 5], name='input_x')
    y_target = tf.placeholder(tf.float32, [None,1], name='target_y')

    # Graph definition
    y_predicted = tf.matmul(X,W) + b

    # Error definition
    meanSquaredError = tf.reduce_mean(tf.reduce_mean(tf.square(y_predicted - y_target), 
                                                reduction_indices=1, 
                                                name='squared_error'), 
                                  name='mean_squared_error')

    # Training mechanism
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
    train = optimizer.minimize(loss=meanSquaredError)
    return W, b, X, y_target, y_predicted, meanSquaredError, train

    
def runMult(dataPath, targetPath):

    # Build computation graph
    W, b, X, y_target, y_predicted, meanSquaredError, train = buildGraph()

    # Loading my data
    trainData, trainTarget, testData, testTarget = loadData(dataPath, targetPath)

    # Initialize session
    init = tf.global_variables_initializer()

    sess = tf.InteractiveSession()
    sess.run(init)

    initialW = sess.run(W)  
    initialb = sess.run(b)

    print("Initial weights: %s, initial bias: %.2f"%(initialW, initialb))
    # Training model
    wList = []
    for step in range(0,201):
        _, err, currentW, currentb, yhat = sess.run([train, meanSquaredError, W, b, y_predicted], feed_dict={X: trainData, y_target: trainTarget})
        wList.append(currentW)
        if not (step % 50) or step < 10:        
            print("Iter: %3d, MSE-train: %4.2f, weights: %s, bias: %.2f"%(step, err, currentW.T, currentb))

    # Testing model
    errTest = sess.run(meanSquaredError, feed_dict={X: testData, y_target: testTarget})
    print("Final testing MSE: %.2f"%(errTest))      


if __name__ == '__main__':
    np.set_printoptions(precision=2)
    runMult('./x.npy', './t2.npy')

Initial weights: [[-0.  ]
 [-0.18]
 [-0.18]
 [-0.3 ]
 [ 0.21]], initial bias: 0.00
Iter:   0, MSE-train: 1884.55, weights: [[0.37 0.88 1.97 2.76 3.36]], bias: 0.05
Iter:   1, MSE-train: 216.53, weights: [[0.68 1.43 2.65 3.64 4.43]], bias: 0.07
Iter:   2, MSE-train: 29.68, weights: [[0.85 1.71 2.87 3.9  4.79]], bias: 0.07
Iter:   3, MSE-train: 5.42, weights: [[0.93 1.84 2.95 3.98 4.91]], bias: 0.05
Iter:   4, MSE-train: 1.77, weights: [[0.97 1.9  2.98 4.   4.95]], bias: 0.04
Iter:   5, MSE-train: 1.13, weights: [[0.98 1.93 2.99 4.01 4.97]], bias: 0.02
Iter:   6, MSE-train: 0.99, weights: [[0.99 1.95 3.   4.01 4.98]], bias: 0.00
Iter:   7, MSE-train: 0.93, weights: [[0.99 1.95 3.   4.01 4.98]], bias: -0.02
Iter:   8, MSE-train: 0.90, weights: [[1.   1.96 3.   4.01 4.98]], bias: -0.04
Iter:   9, MSE-train: 0.86, weights: [[1.   1.96 3.   4.01 4.98]], bias: -0.05
Iter:  50, MSE-train: 0.19, weights: [[1.   1.98 3.   4.   4.99]], bias: -0.56
Iter: 100, MSE-train: 0.03, weights: [[1.   1.99 